<a href="https://colab.research.google.com/github/manoj-b-22/IDS-in-IoV/blob/main/code/CNN-LSTM-intra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from keras.layers import Conv1D, Dense, MaxPooling1D, LSTM
from sklearn.metrics import classification_report, confusion_matrix

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
data = pd.read_csv('/content/drive/MyDrive/intravehicle.csv')

In [6]:
y_data = data.pop('4')
x_data = data

In [7]:
train_x,test_x,train_y,test_y = train_test_split(x_data, y_data, test_size=0.2, random_state=0, shuffle=True) 

In [8]:
train_x = train_x.reset_index(drop=True)
test_x = test_x.reset_index(drop=True) 
train_y = train_y.reset_index(drop=True)
test_y = test_y.reset_index(drop=True)

In [9]:
scaler = StandardScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)

In [10]:
encoder = LabelEncoder()
train_y = encoder.fit_transform(train_y)
test_y = encoder.transform(test_y)

In [11]:
model = Sequential()
model.add(Conv1D(activation="tanh", input_shape=(len(x_data.columns),1), kernel_size=2, filters=100))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100, activation="tanh", recurrent_activation="sigmoid"))
model.add(Dense(4, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 3, 100)            300       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1, 100)           0         
 )                                                               
                                                                 
 lstm (LSTM)                 (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 81,104
Trainable params: 81,104
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.fit(train_x, train_y, batch_size=32, epochs=6)

Epoch 1/6
414237/414237 [==============================] - 1785s 4ms/step - loss: 0.1015 - accuracy: 0.9473
Epoch 2/6
414237/414237 [==============================] - 1516s 4ms/step - loss: 0.0839 - accuracy: 0.9555
Epoch 3/6
414237/414237 [==============================] - 1512s 4ms/step - loss: 0.0799 - accuracy: 0.9571
Epoch 4/6
414237/414237 [==============================] - 1549s 4ms/step - loss: 0.0778 - accuracy: 0.9581
Epoch 5/6
414237/414237 [==============================] - 1889s 5ms/step - loss: 0.0763 - accuracy: 0.9589
Epoch 6/6
414237/414237 [==============================] - 1574s 4ms/step - loss: 0.0752 - accuracy: 0.9595


In [13]:
pred = model.predict(test_x, verbose=1)
pred_y = np.argmax(pred, axis=1)

103560/103560 [==============================] - 173s 2ms/step


In [14]:
confusion_matrix(test_y, pred_y)

array([[ 117497,       0,     230,       0],
       [      0,   77398,   20804,       0],
       [ 102146,   12956, 2732371,     437],
       [      0,       0,       0,  250056]])

In [15]:
report = classification_report(test_y, pred_y, digits=6)
print(report)

              precision    recall  f1-score   support

           0   0.534945  0.998046  0.696547    117727
           1   0.856608  0.788151  0.820955     98202
           2   0.992361  0.959430  0.975618   2847910
           3   0.998255  1.000000  0.999127    250056

    accuracy                       0.958788   3313895
   macro avg   0.845542  0.936407  0.873062   3313895
weighted avg   0.972533  0.958788  0.962894   3313895

